In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, time, shutil, sys
#from pathlib import Path
from glob import glob
from pprint import pprint
import numpy as np
import sklearn as sk
import spikeinterface.full as si
import matplotlib.pyplot as plt
import warnings

sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import spike_sorting as ss
from axon_tracking import template_extraction as te

In [ ]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/" # Fixed path root that all recordings have in common
path_pattern = ["SCR_rebuttal_week_*", "2*", "M0*","AxonTracking","0*"] # Variable part of the path, where we collect all possible combinations using wildcards (*). It is still recommended to be as specific as possible to avoid ambiguities.
file_name = "data.raw.h5" # File name of the recording

full_path = os.path.join(root_path, *path_pattern, file_name)
path_list = glob(full_path)
print(f'Found {len(path_list)} recording paths matching the description:\n{full_path}\n')

In [ ]:
save_path_changes = {'pos': [0, 6, 7, -1, -2], 'vals': ['/', 'intermediate_data', 'Mea1k', '','']}
save_path = ss.convert_rec_path_to_save_path(full_path, save_path_changes)
    
print(f'The save path corresponds to the pattern:\n {save_path}\n')

In [ ]:
sorting_dict = te.find_successful_sortings(path_list, save_path_changes)
print(f'Found {sum(map(len, sorting_dict.values()))} successful sortings')

In [ ]:
te_params = dict()
te_params['align_cutout'] = True
te_params['upsample'] = 2
te_params['rm_outliers'] = True
te_params['n_jobs'] = 16
te_params['n_neighbors'] = 10
te_params['peak_cutout'] = 2 #Looking for peak +- this value around the expected peak (removing minor offsets)
te_params['overwrite'] = False

qc_params = dict()
qc_params['min_n_spikes'] = 50
qc_params['exclude_mua'] = True

In [ ]:
#To suppress warning when the outlier detection has too few samples
warnings.filterwarnings("ignore", message="greater than the total number of samples")

te.extract_templates_from_sorting_dict(sorting_dict, qc_params, te_params)